In [1]:
import polars as pl
from polars import col as c
import altair as alt
from camminapy.plot import altair_theme

altair_theme()

In [2]:
df = pl.read_parquet("../data/data.parquet")

In [7]:
df.filter(pl.col("Rank") == 1).sort("Date").write_csv("../data/worldrecords.csv")

Rank,Mark,Competitor,DOB,Nat,Pos,Venue,Date,Results Score,Mark [metres or seconds],Event,Wind,Sex
i64,str,str,date,str,str,str,date,i64,f64,str,f64,str
1,"""2:03:06""","""Daniel BAUTIST…",1952-08-04,"""MEX""","""1""","""Cherkassy (URS…",1980-04-27,1227,7386.0,"""30 Kilometres …",null,"""male"""
1,"""1:53.28""","""Jarmila KRATOC…",1951-01-26,"""TCH""","""1""","""München (GER)""",1983-07-26,1286,113.2,"""800 Metres""",null,"""female"""
1,"""47.6""","""Marita KOCH""",1957-02-18,"""GDR""","""1""","""Bruce Stadium,…",1985-10-06,1304,47.0,"""400 Metres""",null,"""female"""
1,"""1:12.81""","""Johnny GRAY""",1960-06-19,"""USA""","""1""","""Santa Monica, …",1986-05-24,1239,72.8,"""600 Metres""",null,"""male"""
1,"""74.08""","""Jürgen SCHULT""",1960-05-11,"""GDR""","""1""","""Neubrandenburg…",1986-06-06,1320,74.0,"""Discus Throw""",null,"""male"""
1,"""86.74""","""Yuriy SEDYKH""",1955-06-11,"""URS""","""1""","""Neckarstadion,…",1986-08-30,1307,86.7,"""Hammer Throw""",null,"""male"""
1,"""22.63""","""Natalya LISOVS…",1962-07-16,"""URS""","""1.0""","""Moskva (URS)""",1987-06-07,1372,22.6,"""Shot Put""",null,"""female"""
1,"""2.09""","""Stefka KOSTADI…",1965-03-25,"""BUL""","""1""","""Stadio Olimpic…",1987-08-30,1309,2.0,"""High Jump""",null,"""female"""
1,"""7.52""","""Galina CHISTYA…",1962-07-26,"""URS""","""1""","""Leningrad (URS…",1988-06-11,1333,7.5,"""Long Jump""",1.4,"""female"""
